In [1]:
import sys
sys.path.insert(0, '../../')

import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import topo
import typing
import utils
import yaml
from tqdm.notebook import tqdm

matplotlib.rc('font', family='Times New Roman', size=16)

In [2]:
def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

def gen_graphs(graph_count, source_selector_dict):
    source_selector = graph.MultiDomainSourceSelector(source_selector_dict)
    gen_args_list = [
            {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 3),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
            "sources_num": random.randint(1, 3),
        }
        for _ in range(graph_count)
    ]
    return [
        graph.MultiSourceGraphGenerator("g" + str(idx), **gen_args).gen_dag_graph()
        for idx, gen_args in enumerate(gen_args_list)
    ]

In [3]:
from tqdm.notebook import trange

def big_avg(array):
    return sum(array) / len(array)

def run(sc, labels, source_selector_dict):
    flow_bp_data = [[] for _ in labels]
    all_cloud_bp_data = [[] for _ in labels]
    sbon_bp_data = [[] for _ in labels]
    flow_event_data = [[] for _ in labels]
    all_cloud_event_data = [[] for _ in labels]
    sbon_event_data = [[] for _ in labels]
    edge_random_event_data = [[] for _ in labels]
    test_round = 100
    for idx, graph_count in enumerate(labels):
        print(graph_count)
        for _ in trange(test_round, desc="running with {} graphs".format(graph_count)):
            graph_list = gen_graphs(graph_count, source_selector_dict)
            sc.topo.clear_occupied()
            flow_scheduler = sch.MultiEdgeFlowScheduler(sc)
            flow_calculator = sch.LatencyCalculator(sc.topo)
            flow_result_list = flow_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, flow_result_list):
                assert result is not None
                flow_calculator.add_scheduled_graph(g, result)
            flow_latency, flow_bp, _ = flow_calculator.compute_latency()
            for g in graph_list:
                flow_event_data[idx].append(flow_latency[g.uuid] / g.number_of_vertices())
                flow_bp_data[idx].append(flow_bp[g.uuid])
            # flow_event_data[idx].extend()

            # sc.topo.clear_occupied()
            # sbon_scheduler = sch.SBONScheduler(sc)
            # sbon_calculator = sch.LatencyCalculator(sc.topo)
            # sbon_result_list = sbon_scheduler.schedule_multiple(graph_list)
            # for g, result in zip(graph_list, sbon_result_list):
            #     assert result is not None
            #     sbon_calculator.add_scheduled_graph(g, result)
            # sbon_latency, sbon_bp = sbon_calculator.compute_latency()
            # for g in graph_list:
            #     sbon_event_data[idx].append(sbon_latency[g.uuid] / g.number_of_vertices())
            #     sbon_bp_data[idx].append(sbon_bp[g.uuid])

            # sc.topo.clear_occupied()
            # all_cloud_scheduler = sch.AllCloudScheduler(sc)
            # all_cloud_calculator = sch.LatencyCalculator(sc.topo)
            # all_cloud_result_list = all_cloud_scheduler.schedule_multiple(graph_list)
            # for g, result in zip(graph_list, all_cloud_result_list):
            #     assert result is not None
            #     all_cloud_calculator.add_scheduled_graph(g, result)
            # all_cloud_latency, all_cloud_bp = all_cloud_calculator.compute_latency()
            # for g in graph_list:
            #     all_cloud_event_data[idx].append(all_cloud_latency[g.uuid] / g.number_of_vertices())
            #     all_cloud_bp_data[idx].append(all_cloud_bp[g.uuid])
            # all_cloud_event_data[idx].extend(list(all_cloud_latency.values()))

            # sc.topo.clear_occupied()
            # edge_random_scheduler = sch.EdgeRandomScheduler(sc)
            # edge_random_calculator = sch.LatencyCalculator(sc.topo)
            # edge_random_result_list = edge_random_scheduler.schedule_multiple(graph_list)
            # for g, result in zip(graph_list, edge_random_result_list):
            #     assert result is not None
            #     edge_random_calculator.add_scheduled_graph(g, result)
            # edge_random_latency, edge_random_bp = edge_random_calculator.compute_latency()
            # edge_random_event_data[idx].extend(list(edge_random_latency.values()))

            # for k in all_cloud_latency.keys():
                # flow_event_data[idx].append(flow_latency[k] / all_cloud_latency[k])
                # edge_random_event_data[idx].append(edge_random_latency[k] / flow_latency[k])
    flow_event_data = [big_avg(i) for i in flow_event_data]
    flow_bp_data = [big_avg(i) for i in flow_bp_data]
    # all_cloud_event_data = [big_avg(i) for i in all_cloud_event_data]
    # all_cloud_bp_data = [big_avg(i) for i in all_cloud_bp_data]
    # sbon_event_data = [big_avg(i) for i in sbon_event_data]
    # sbon_bp_data = [big_avg(i) for i in sbon_bp_data]
    # return flow_event_data, flow_bp_data, all_cloud_event_data, all_cloud_bp_data, sbon_event_data, sbon_bp_data
    return flow_event_data, flow_bp_data
    # ax_lat.plot(flow_event_data)
    # ax_lat.plot(all_cloud_event_data)
    # ax_bp.plot(flow_bp_data)
    # ax_bp.plot(all_cloud_bp_data)
    # plt.eventplot(flow_event_data, lineoffsets=[i-width for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C0", orientation="vertical")
    # plt.eventplot(edge_random_event_data, lineoffsets=[i for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C1", orientation="vertical")
    # plt.eventplot(all_cloud_event_data, lineoffsets=[i+width for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C2", orientation="vertical")
    # plt.violinplot(flow_event_data, vert=True, positions=[i-width for i in range(len(labels))], widths=width)
    # plt.violinplot(edge_random_event_data, vert=True, positions=[i for i in range(len(labels))], widths=width)
    # plt.violinplot(all_cloud_event_data, vert=True, positions=[i+width for i in range(len(labels))], widths=width)

In [7]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e6h.yaml", "r").read(), Loader=yaml.Loader))
f_lat1, f_bp1 = run(sc, [4, 8, 12, 16, 20], 
                    {"edge0": {'e0rasp1': 10, 'e0rasp2': 10, 'e0rasp3': 10},
                    "edge1": {'e1rasp1': 10, 'e1rasp2': 10, 'e1rasp3': 10}})

# sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e18h.yaml", "r").read(), Loader=yaml.Loader))
# edge0 = {"e0rasp"+str(i): 8 for i in range(1, 7)}
# edge1 = {"e1rasp"+str(i): 8 for i in range(1, 4)}
# edge1.update({"e1vm"+str(i): 16 for i in range(1, 4)})
# edge2 = {"e2rasp"+str(i): 8 for i in range(1, 5)}
# edge2.update({"e2vm"+str(i): 16 for i in range(1, 3)})
# source_dict = {"edge0": edge0, "edge1": edge1, "edge2": edge2}
# f_lat2, f_bp2 = run(sc, [40, 60, 80, 100, 120], source_dict)

print(f_lat1)
# print(f_lat2)

4


running with 4 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

8


running with 8 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

12


...

16


running with 16 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

20


running with 20 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

[5.032409570517775, 6.188456727393897, 7.294165268550158, 8.447114757515429, 9.617828394874445]
